# Jedgment Summarization



## 安裝套件


In [1]:
! pip install transformers
! pip install datasets
! pip install torcheval
! pip install scikit-learn
! pip install torch
! pip install pytorch-ignite
! pip install ipywidgets
! pip install jieba
! pip install opencc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
import transformers as T
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from ignite.metrics import Rouge
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import jieba
import os

from opencc import OpenCC

s2t_converter = OpenCC('s2t.json')
t2s_converter = OpenCC('t2s.json')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## 超參數


In [3]:
lr = 5e-5
epochs = 30
train_batch_size = 1
validation_batch_size = 1
test_batch_size = 1
random_seed = 42
# max_length = 300
accumulation_steps = 16

# path setting that souldn't be changed
tokenizer_path = "./saved_tokenizer"
dataset_path = "./train_clean.xlsx"
model_path = "./saved_model"

if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(tokenizer_path):
    os.mkdir(tokenizer_path)

In [4]:
data_df = pd.read_excel(dataset_path)
data_df.head()

,Unnamed: 0.1,Unnamed: 0,期數,出刊日,案件類型,類型,頁數,裁判字號,爭點,裁判日期,裁判案由,摘要,裁判原文,共同長度
0,0,解讀裁判周報,42,201802,刑事法,程序法,45-46,最高法院106年度台非字第155號刑事判決,沒入具保人繳納之保證金，係對被告逃匿後，所給予具保人之制裁，自應以被告確實在逃匿中為其要件。...,106年8月23日,詐欺沒入保證金,（一）具保之被告逃匿者，應命具保人繳納指定之保證金額，並沒入之。不繳納者，強制執行。保證金已...,判決如下：主文上訴駁回。理由一、本件非常上訴理由稱：「按刑事訴訟法第118條之沒入保證金，...,519
1,1,解讀裁判周報,28,201612,公法,勞工法,52-53,最高法院105年台上字第1528號民事判決,民法第483條之1規定，受僱人服勞務，其生命、身體、健康有受危害之虞者，僱用人應按其情形為必...,105年9月8日,請求給付職災補償等,"查被上訴人所受增加生活上需要、勞動能力喪失、精神慰撫金之損害共計1,270萬6,671元，其...",判決如下：主文原判決關於命上訴人給付被上訴人新台幣肆佰捌拾伍萬玖仟捌佰捌拾貳元本息及該訴訟費...,586
2,2,解讀裁判周報,29,201701,刑事法,實體法,28-29,最高法院105年度台上字第499號刑事判決,政府採購法第34條第2項規定，機關辦理採購，不得於開標前洩漏底價，領標、投標廠商之名稱與家數...,105年2月26日,違反貪污治罪條例,政府採購法第34條第2項規定，機關辦理採購，不得於開標前洩漏底價，領標、投標廠商之名稱與家數...,請託透露投標廠商名稱之際，未予斷然拒絕，且回覆劉吉祥稱：會儘量幫忙等語，甚至將部分廠商投標文...,432
3,3,解讀裁判周報,22,201606,公法,行政法,53-55,最高行政法院105年度判字第160號判決,有關法規命令之發布，行政機關得否僅以在網站上公告之方式為之？,105年4月8日,政府採購法,一、 政府採購法第31條第2項第8款即為立法者於立法定制時，基於政府採購法規範之採購事項的複...,」，固應就該項法律整體所表現之關聯意義為判斷，而非拘泥於特定法條之文字，然「依此種『概括授權...,1466
4,4,解讀裁判周報,33,201705,公法,行政法,72-75,最高行政法院106年度判字第170號判決,行政法院就廢棄物清理法第27條第11款授權訂定之法規命令為具體法規審查。,106年4月7日,廢棄物清理法,一、 廢棄物清理法第1條前段揭示該法立法目的為「有效清除、處理廢棄物，改善環境衛生，維護國民...,設置行為為污染行為，概予禁止並處罰，已逾越母法授權範圍，與法律保留原則尚有未符。應自本解釋公...,2136


## 載入BART模型
 - 使用huggingface裝載模型的架構、參數和tokenizer
 - 保存在路徑./cache/中
 - 用.to(device)把模型裝載入訓練設備(GPU)
 - 使用的 tokenizer 請儲存到指定路徑

In [5]:
from transformers import PegasusForConditionalGeneration
max_encoder_position_embeddings = 512
max_decoder_position_embeddings = 512

model = T.BartForConditionalGeneration.from_pretrained("IDEA-CCNL/Randeng-BART-139M-SUMMARY")
tokenizer = T.AutoTokenizer.from_pretrained("IDEA-CCNL/Randeng-BART-139M-SUMMARY")
# tokenizer.save_pretrained(tokenizer_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/859k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

In [6]:
model = model.to(device)
optimizer = AdamW(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

## 資料處理
 - 使用 torch.utils.data 中的 Dataset 和 Dataloader 成批次地讀取和預處理資料

In [7]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = tokenizer.batch_encode_plus([each["origin_context"] for each in sample], max_length=max_encoder_position_embeddings, padding=True, truncation=True, return_tensors="pt")
    model_outputs = tokenizer.batch_encode_plus([each["summary"] for each in sample], max_length=max_decoder_position_embeddings, padding=True, truncation=True, return_tensors="pt")
    return model_inputs["input_ids"].to(device), model_outputs["input_ids"].to(device), [each["index"] for each in sample]

class Legal_Judgment_Dataset(Dataset):
    def __init__(self, data_df) -> None:
        super().__init__()

        self.data = []
        for index, row in data_df.iterrows():
            # 都轉為簡體字
            origin_context = t2s_converter.convert("summary: " + str(row['裁判原文']))
            summary = t2s_converter.convert(str(row['摘要']))
            self.data.append({"origin_context": origin_context, "summary": summary, "index": index})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)


- 在範例程式中有切分成 train, valid, test，實作上可將全部資料用於 train 與 valid

#### split into train and valid

In [8]:
# dataset = Legal_Judgment_Dataset(data_df)
# train_dataset, validation_dataset = train_test_split(dataset, test_size=0.2, random_state=random_seed)
# summary_train = DataLoader(train_dataset, collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
# summary_validation = DataLoader(validation_dataset, collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)

# rouge = Rouge(variants=["L", 2], multiref="best")
# print(f"Dataset example: \n{train_dataset[0]} \n{train_dataset[1]} \n{train_dataset[2]}")

#### Not split

In [9]:
dataset = Legal_Judgment_Dataset(data_df)
summary_train = DataLoader(dataset, collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)

rouge = Rouge(variants=["L", 2], multiref="best")
print(f"Dataset example: \n{dataset[0]} \n{dataset[1]} \n{dataset[2]}")

Dataset example: 
{'origin_context': 'summary: 判决如下：主文上诉驳回。理由一、本件非常上诉理由称：「\uf6b0按刑事诉讼法第118条之没入保证金，系因被告于具保人以相当金额具保后逃匿，所给予具保人之制裁；虽与刑法从刑之没收有别，但此项依刑事诉讼法所为之没入裁定，依同法第470条第2项规定，既具有民事强制执行名义之效力，即与判决有同一效力，如有违背法令，自得提起非常上诉。再具保之被告逃匿者，应命具保人缴纳指定之保证金额，并没入之；不缴纳者，强制执行；保证金已缴纳者，没入之，刑事诉讼法第118条定有明文。是没入具保人缴纳之保证金，应以被告在逃匿中为其要件。具保之被告虽曾逃匿，但已经缉获归案，即不得再以被告逃匿为由而裁定没入具保人缴纳之保证金。（最高法院99年度台非字第336号刑事裁判参见）\uf6af经查本件被告李韦达于台湾高雄地方法院检察署（下称高雄地检）100年度侦字第24109号等诈欺案件侦查期间，由具保人吴宜蓁以新台币（下同）10万元为其具保，嗣该案移送台湾高雄地方法院101年度易字第698号案件审理，经判处有期徒刑6年2月，再经台湾高等法院高雄分院以102年度上易字第546号判决驳回上诉确定，并送高雄地检执行。兹因被告经高雄地检检察官合法传唤、拘提，无正当理由不到案执行，具保人经合法通知，亦未遵期使被告到案执行。嗣又经高雄地检函请台湾屏东地方法院检察署（下称屏东地检）检察官代为执行，亦经屏东地检检察官合法传唤、拘提，无正当理由不到案执行，具保人经合法通知亦未遵期使被告到案执行，高雄地检检察官乃于103年12月31日向法院声请没入上开保证金，并于104年1月8日以雄检瑞峨缉字第26号通缉书通缉被告。上开没入保证金声请，经台湾高雄地方法院于104年1月15日以104年度声字第182号裁定没入保证金，具保人依法提起抗告，复经台湾高等法院高雄分院于104年3月9日以原裁定抗告驳回确定。惟被告早于104年2月25日为警缉获到案，并经高雄地检检察官于同日指挥发监执行。有高雄地检103年执声没字第278号执行卷宗、高雄市政府警察局鼓山分局104年2月25日高市警鼓分侦字第00000000000号解送人犯报告书、高雄地检104年2月25日乙种执行指挥书、被告完整矫正简表、被告在监在押纪录表等附卷可稽。被告虽在该案执行中，因逃匿

## 驗證
驗證程式
 - 將驗證資料輸入模型，用Rouge-2評價輸出的效果
 - Rouge的使用方法參考 https://pytorch.org/ignite/generated/ignite.metrics.Rouge.html

In [10]:
def evaluate(model, dataset, data_df):
    pbar = tqdm(dataset)
    pbar.set_description(f"Evaluating")
    loss_list = []
    for inputs, targets, indices in pbar:
        loss = model(inputs, labels=targets).loss
        loss_list.append(loss.item())
        pbar.set_postfix(loss = loss.item())
        outputs = [each.replace("<unk>", "").replace("<pad>","").replace("</s>","").replace(" ", "") for each in tokenizer.batch_decode(model.generate(inputs, max_length=max_decoder_position_embeddings, min_length=200))]
        targets = [s2t_converter.convert(each.replace("<unk>", "").replace("<pad>","").replace("</s>","").replace(" ", "")) for each in tokenizer.batch_decode(targets)]
        # 轉為繁體
        outputs = [s2t_converter.convert(each) for each in outputs]
        # 真算分數
        # targets = [s2t_converter.convert(data_df.loc[i, "摘要"]) for i in indices]

        print("model generate examples:")
        print(f"output: {outputs}")
        print(f"target: {targets}")
        for out, tar in zip(outputs, targets):
            sentence = " ".join(jieba.cut(out)).split()
            ground_truth = " ".join(jieba.cut(tar)).split()
            for s in sentence:
                rouge.update(([s], [ground_truth]))

    return rouge.compute(), np.mean(np.array(loss_list))

In [11]:
def split_text(text, max_length, step):
    tokens = list(text)
    chunks = []
    if len(tokens) < 2000:

        if (len(tokens) <= 500):
            chunks.append(t2s_converter.convert("summary: " + "".join(tokens)))

        while(len(tokens) > 500):
            chunks.append(t2s_converter.convert("summary: " + "".join(tokens[:500])))
            tokens = tokens[500:]

            if len(tokens) < 250:
                break
            elif len(tokens) <= 500:
                chunks.append(t2s_converter.convert("summary: " + "".join(tokens[:500])))

    else:
        chunks = [
            t2s_converter.convert("summary: " + "".join(tokens[:500])),
            t2s_converter.convert("summary: " + "".join(tokens[-1500:-1000])),
            t2s_converter.convert("summary: " + "".join(tokens[-1000:-500])),
            t2s_converter.convert("summary: " + "".join(tokens[-500:]))
        ]
    return chunks

def evaluate_with_sliding_window(model, dataset, data_df):
    pbar = tqdm(dataset)
    pbar.set_description(f"Evaluating")

    for _, targets, indices in pbar:

        chunks = split_text(data_df.loc[indices[0], "裁判原文"], 500, 500)
        inputs = tokenizer(chunks, padding=True, truncation=True, max_length=max_encoder_position_embeddings, return_tensors="pt")
        inputs = inputs["input_ids"].to(device)

        outputs = [each.replace("<unk>", "").replace("<pad>","").replace("</s>","").replace("<s>","").replace(" ", "") for each in tokenizer.batch_decode(model.generate(inputs, max_length=max_decoder_position_embeddings, min_length=100))]
        outputs = [ "".join(outputs) ]

        targets = [s2t_converter.convert(each.replace("<unk>", "").replace("<pad>","").replace("</s>","").replace("<s>","").replace(" ", "")) for each in tokenizer.batch_decode(targets)]
        # 轉為繁體
        outputs = [s2t_converter.convert(each) for each in outputs]
        # 真算分數
        # targets = [s2t_converter.convert(data_df.loc[i, "摘要"]) for i in indices]

        print("model generate examples:")
        print(f"output: {outputs}")
        print(f"target: {targets}")
        for out, tar in zip(outputs, targets):
            sentence = " ".join(jieba.cut(out)).split()
            ground_truth = " ".join(jieba.cut(tar)).split()
            for s in sentence:
                rouge.update(([s], [ground_truth]))

    return rouge.compute()

## 訓練
 - 將資料成批次輸入BART模型，並獲取其損失函數數值，隨後計算梯度優化
 - tqdm用來顯示模型的訓練進度

In [12]:
import warnings
import logging

# Suppress all warnings using the warnings module
warnings.filterwarnings("ignore")# Suppress specific warning types (optional)
warnings.filterwarnings("ignore", category=UserWarning)
# Suppress all other loggers
logging.getLogger().setLevel(logging.ERROR)

In [13]:
for ep in range(epochs):
    model.train()
    optimizer.zero_grad()
    pbar = tqdm(summary_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for i, (inputs, targets, _) in enumerate(pbar):

        loss = model(input_ids=inputs, labels=targets).loss / accumulation_steps
        loss.backward()

        if (i+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        pbar.set_postfix(loss = loss.item())
    # torch.save(model, f'{model_path}/ep{ep}.mod')

    validation_score, validation_loss = None, None

    # if (ep+1) % 10 == 0:

        # model.eval()
        # validation_score = evaluate_with_sliding_window(model, summary_validation, data_df)

    #     model.eval()
    #     validation_score, validation_loss = evaluate(model, summary_validation, data_df)

    print(f"Rouge-2 score on epoch {ep}:", validation_score)

model.eval()

pbar = tqdm(summary_train)
for i, (inputs, targets, _) in enumerate(pbar):

    outputs = [each.replace("<unk>", "").replace("<pad>","").replace("</s>","").replace(" ", "") for each in tokenizer.batch_decode(model.generate(inputs, max_length=max_decoder_position_embeddings, min_length=200))]
    targets = [s2t_converter.convert(each.replace("<unk>", "").replace("<pad>","").replace("</s>","").replace(" ", "")) for each in tokenizer.batch_decode(targets)]
    # 轉為繁體
    outputs = [s2t_converter.convert(each) for each in outputs]
    print("model generate examples:")
    print(f"output: {outputs}")
    print(f"target: {targets}")

torch.save(model, f'{model_path}/ep{ep}.mod')

  0%|          | 0/230 [00:00<?, ?it/s]

Rouge-2 score on epoch 0: None


  0%|          | 0/230 [00:00<?, ?it/s]

Rouge-2 score on epoch 1: None


  0%|          | 0/230 [00:00<?, ?it/s]

...

Rouge-2 score on epoch 28: None


  0%|          | 0/230 [00:00<?, ?it/s]

Rouge-2 score on epoch 29: None


  0%|          | 0/230 [00:00<?, ?it/s]

model generate examples:
output: ['刑事訴訟法第15條復明定一定關係之人得於偵查、審判中陪同在場及陳述意見,此之陪同人,除與被害人具有親屬關係者外,尚包括法律社會工作者之社工人員、輔導人員、醫師及心理師等專業人士在內;陪同在場具有穩定及緩和被害人不安與緊張之情緒,避免受到二度傷害,而法律社會工作機制之介入,並著重在藉由心理諮商或精神醫學等專業,以佐證被害人證詞之有效性或憑信性,兼負有協助偵、審機關發見真實之義務與功能,與外國法制之專家證人同其作用。因此,社工或輔導人員就其所介入輔導個案,經過直接觀察及以個人實際經驗爲基礎所爲之書面或言詞陳述,即屬於見聞經過之證人性質,而醫療或心理衛生人員針對被害人於治療過程中,所產生之與待證事實相關之反應或身心狀況(如有無罹患創傷後壓力症候羣或相關精神、心理疾病)所提出之意見,或以其經驗及訓練就通案之背景資訊陳述專業意見,以供法院參佐,則爲鑑定證人或鑑定人身分。凡此,均屬與被害人陳述不具同一性之獨立法定證據方法,得供爲判斷被害人陳述憑信性之補強證據,自應分辨其身分而爲適當之具結。']
target: ['刑事訴訟法第158條之3規定「證人、鑑定人依法應具結而未具結者,其證言或鑑定意見,不得作爲證據。」所謂鑑定,系由法院或檢察官選任之鑑定人或囑託之鑑定機構,除憑藉其特別知識經驗,就特定物(書)證加以鑑(檢)驗外,並得就無關親身經歷之待鑑事項,僅依憑其特別知識經驗而陳述或報告其專業意見,以供法院審判之參酌依據,具有可替代性。至於刑事訴訟法第210條規定「訊問依特別知識得知已往事實之人者,適用關於人證之規定。」此即學理所稱之「鑑定證人」,其既系依特別知識而得知親身經歷之已往事實,因有其不可替代之特性,故法律明定應適用關於人證之規定。兩者因其證據性質不同,爲確保其真實性,所爲結文各有不同,前者依刑事訴訟法第202條規定,結文內應記載「必爲公正誠實之鑑定」等語,後者適用同法第189條第1項規定,結文應記載「當據實陳述,決無匿、飾、增、減」等語。鑑定證人,因具證人與鑑定人二種身分,然所陳述者,既系已往見聞經過之事實,具有不可替代性,自不失爲證人,適用關於人證之規定;惟如所陳述者或並在使依特別知識,就所觀察之現在事實報告其判斷之意見,仍爲鑑定人。於此,應分別情形命具證人結,或加具鑑定人結。其人究屬證人或鑑定人,自

## 測試
- 使用以下程式接跑 test data

In [14]:
# model = torch.load("/content/saved_model/ep29.mod")

In [15]:
test_dataset_path = './test_clean.xlsx'
origin_test_path = "./test.xlsx"

In [16]:
test_df = pd.read_excel(test_dataset_path)
test_df.head()

,Unnamed: 0.1,Unnamed: 0,期數,出刊日,案件類型,類型,頁數,裁判字號,爭點,裁判日期,裁判案由,裁判原文
0,0,解讀裁判周報,41,201801,公法,智慧財產權法,70-73,最高行政法院106年度判字第651號行政判決,專利進步性審查時，應如何判斷該發明所屬技術領域中具有通常知識者是否有動機能結合複數引證之技術...,106年11月22日,新型專利舉發,螺孔，該側接面係於該外緣面的兩端側設置，該側接面遠離該外緣面的一端側係與該本體的表面連接；兩...
1,1,解讀裁判周報,30,201702,民事法,實體法,2024-09-10 00:00:00,最高法院105年度台上字第1250號民事判決,所謂被害人與有過失，係指被害人苟能盡其善良管理人之自我注意，即得避免其損害之發生或擴大，竟不...,105年7月27日,請求確認債權不存在,判決如下：主文原判決除假執行部分外廢棄，發回台灣高等法院。理由本件上訴人主張：兩造於民國九十...
2,2,解讀裁判周報,38,201710,刑事法,實體法,36-36,最高法院106年度台上字第675號刑事判決,正當防衛必須對於現在不法之侵害始得為之，而互毆係屬多數動作構成單純一罪而互為攻擊之傷害行為，...,106年3月22日,殺人,刑事訴訟法第三百七十七條規定，上訴於第三審法院，非以判決違背法令為理由，不得為之。是提起第三...
3,3,解讀裁判周報,23,201607,公法,行政法,60-62,最高行政法院105年度判字第168號判決,文化資產保存法第12條規定：「主管機關應普查或接受個人、團體提報具古蹟、歷史建築、聚落價值建...,105年4月14日,文化資產保存,以討論，且扣除該委員，其餘4位委員亦仍表示不建議列冊追踪等語，資為抗辯，求為判決駁回上訴人之...
4,4,解讀裁判周報,38,201710,刑事法,實體法,33-34,最高法院106年度台上字第539號刑事判決,於二人以上共同實行犯罪之情形，固基於責任共同原則，共同正犯應就全部犯罪結果負其責任，然於集團...,106年4月7日,違反臺灣地區與大陸地區人民關係條例,判決如下：主文原判決撤銷，發回台灣高等法院高雄分院。理由一、本件原判決認定被告林寬達（原名林...


In [17]:
origin_test_df = pd.read_excel(origin_test_path)
origin_test_df.head()

,Unnamed: 0,期數,出刊日,案件類型,類型,頁數,裁判字號,爭點,裁判日期,裁判案由,裁判原文
0,解讀裁判周報,41,201801,公法,智慧財產權法,70-73,最高行政法院106年度判字第651號行政判決,專利進步性審查時，應如何判斷該發明所屬技術領域中具有通常知識者是否有動機能結合複數引證之技術...,106年11月22日,新型專利舉發,最高行政法院判決106年度判字第651號上訴人鑫茂機械工業股份有限公司代表人劉金源訴訟代理人...
1,解讀裁判周報,30,201702,民事法,實體法,2024-09-10 00:00:00,最高法院105年度台上字第1250號民事判決,所謂被害人與有過失，係指被害人苟能盡其善良管理人之自我注意，即得避免其損害之發生或擴大，竟不...,105年7月27日,請求確認債權不存在,最高法院民事判決一○五年度台上字第一二五○號上訴人任順律師即榮電股份有限公司破產管理人參加人...
2,解讀裁判周報,38,201710,刑事法,實體法,36-36,最高法院106年度台上字第675號刑事判決,正當防衛必須對於現在不法之侵害始得為之，而互毆係屬多數動作構成單純一罪而互為攻擊之傷害行為，...,106年3月22日,殺人,最高法院刑事判決一○六年度台上字第六七五號上訴人張道宗選任辯護人李宗炎律師上列上訴人因殺人案...
3,解讀裁判周報,23,201607,公法,行政法,60-62,最高行政法院105年度判字第168號判決,文化資產保存法第12條規定：「主管機關應普查或接受個人、團體提報具古蹟、歷史建築、聚落價值建...,105年4月14日,文化資產保存,最高行政法院判決105年度判字第168號上訴人台灣環境保護聯盟代表人劉俊秀訴訟代理人吳君婷律...
4,解讀裁判周報,38,201710,刑事法,實體法,33-34,最高法院106年度台上字第539號刑事判決,於二人以上共同實行犯罪之情形，固基於責任共同原則，共同正犯應就全部犯罪結果負其責任，然於集團...,106年4月7日,違反臺灣地區與大陸地區人民關係條例,最高法院刑事判決一○六年度台上字第五三九號上訴人台灣高等法院高雄分院檢察署檢察官被告林寬達（...


In [18]:
delete_idx = 33
origin_test_df.loc[delete_idx, "裁判原文"]

'最高行政法院判決106年度判字第441號上訴人蕭興仁訴訟代理人陳建勛律師被上訴人經濟部智慧財產局代表人洪淑敏參加人景岳生物科技股份有限公司代表人陳根德訴訟代理人俞伯璋律師洪聖濠律師陳孚竹專利代理人上列當事人間發明專利舉發事件，上訴人對於中華民國105年5月19日智慧財產法院104年度行專訴字第121號行政判決，關於訴之變更部分提起上訴，本院判決如下：主文上訴駁回。上訴審訴訟費用由上訴人負擔。理由請點選下方附件連結以取得判決全文。中華民國106年8月10日最高行政法院第一庭審判長法官劉鑫楨法官胡方新法官程怡怡法官張國勳法官汪漢卿以上正本證明與原本無異中華民國106年8月10日書記官莊子誼關鍵詞：發明第I284149號專利副乾酪乳桿菌舉發聲明訴訟標的訴之變更追加'

In [19]:
def delete_former_part(text):

    keywords = ["裁定如下", "判決如下", "決定如下"]
    text = str(text)

    for keyword in keywords:
        idx = text.find(keyword)
        if idx >= 0:
            break

    if idx < 0:
        return text
    else:
        return text[idx:]

def delete_latter_part(text):

    keywords = ["判決如主文", "裁定如主文", "決定如主文"]
    text = str(text)

    for keyword in keywords:
        idx = text.find(keyword)
        if idx >= 0:
            break

    if idx < 0:
        return text
    else:
        return text[:idx+5]

In [20]:
def split_text(text, max_length, step):
    tokens = list(text)
    chunks = []
    if len(tokens) < 2000:

        if (len(tokens) <= 500):
            chunks.append(t2s_converter.convert("summary: " + "".join(tokens)))

        while(len(tokens) > 500):
            chunks.append(t2s_converter.convert("summary: " + "".join(tokens[:500])))
            tokens = tokens[500:]

            if len(tokens) < 250:
                break
            elif len(tokens) <= 500:
                chunks.append(t2s_converter.convert("summary: " + "".join(tokens[:500])))

    else:
        chunks = [
            t2s_converter.convert("summary: " + "".join(tokens[:500])),
            t2s_converter.convert("summary: " + "".join(tokens[-1500:-1000])),
            t2s_converter.convert("summary: " + "".join(tokens[-1000:-500])),
            t2s_converter.convert("summary: " + "".join(tokens[-500:]))
        ]
    return chunks

In [21]:
delete_idx = 33
origin_test_df['摘要'] = [ "" for _ in range(len(origin_test_df))]

model.eval()

for i, _ in tqdm(enumerate(origin_test_df["裁判原文"]), total=len(origin_test_df)):

    if i < delete_idx:

        chunks = split_text(str(test_df.loc[i, "裁判原文"]), 500, 500)

    elif i == delete_idx:
        text = str(origin_test_df.loc[i, "裁判原文"])
        text = delete_former_part(text)
        text = delete_latter_part(text)
        chunks = split_text(text, 500, 500)

    else:
        chunks = split_text(str(test_df.loc[i-1, "裁判原文"]), 500, 500)

    inputs = tokenizer(chunks, padding=True, truncation=True, max_length=max_encoder_position_embeddings, return_tensors="pt")
    inputs = inputs["input_ids"].to(device)

    outputs = [each.replace("<unk>", "").replace("<pad>","").replace("</s>","").replace("<s>","").replace(" ", "") for each in tokenizer.batch_decode(model.generate(inputs, max_length=max_decoder_position_embeddings, min_length=100))]
    outputs = "".join(outputs)

    outputs = s2t_converter.convert(outputs)
    origin_test_df.loc[i, "摘要"] = outputs




  0%|          | 0/59 [00:00<?, ?it/s]

In [22]:
# delete_idx = 33
# origin_test_df['摘要'] = [ "" for _ in range(len(origin_test_df))]

# model.eval()

# for i, _ in tqdm(enumerate(origin_test_df["裁判原文"]), total=len(origin_test_df)):

#     if i < delete_idx:
#         text = t2s_converter.convert("summary: " + str(test_df.loc[i, "裁判原文"]))

#     elif i == delete_idx:
#         text = str(origin_test_df.loc[i, "裁判原文"])
#         text = delete_former_part(text)
#         text = delete_latter_part(text)
#         text = t2s_converter.convert("summary: " + text)

#     else:
#         text = t2s_converter.convert("summary: " + str(test_df.loc[i-1, "裁判原文"]))

#     inputs = tokenizer(text, padding=True, truncation=True, max_length=max_encoder_position_embeddings, return_tensors="pt")["input_ids"]
#     inputs = inputs.to(device)
#     outputs = tokenizer.batch_decode(model.generate(inputs, max_length=max_decoder_position_embeddings, min_length=50))
#     outputs = outputs[0].replace("<unk>", "").replace("<pad>","").replace("</s>","").replace("<s>","").replace(" ", "")
#     outputs = s2t_converter.convert(outputs)
#     origin_test_df.loc[i, "摘要"] = outputs

In [23]:
origin_test_df.to_excel("submit.xlsx", index=False)

In [24]:
!pip freeze > requirements.txt